# 1. Install Requirements

In [ ]:
# YOLOv7 and requirements
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

# 2. Download Annotated Dataset

In [ ]:
# Copy Dataset from Roboflow
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="CCByhgZhC987VAUFU6rg")
project = rf.workspace("computer-project").project("scuba-diving-world")
dataset = project.version(1).download("yolov7")

# 3. Model Training

In [ ]:
# YOLOv7 starting checkpoint
%cd /content/yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

In [ ]:
# # Training Command
%cd /content/yolov7
!python train.py --batch 16 --epochs 60 --data {dataset.location}/data.yaml --weights 'yolov7_training.pt' --device 0 


# 4. Export


In [ ]:
#  zip to download weights and results locally

!zip -r export.zip runs/detect
!zip -r export.zip runs/train/exp/weights/best.pt
!zip export.zip runs/train/exp/*

# 5. Convert YOLOv7 weight to TFLite for Android App

In [ ]:
!pip --quiet install onnx onnxruntime onnxsim
!pip install onnx-tf

In [ ]:
%cd /content/yolov7
!python export.py --weights best.pt --grid --end2end --simplify \
--topk-all 100 --iou-thres 0.65 --conf-thres 0.35 --img-size 640 640 --max-wh 640

In [ ]:
!onnx-tf convert -i /content/yolov7/best.onnx -o /content/

In [ ]:
import cv2
import time
import requests
import random
import numpy as np
import onnxruntime as ort
from PIL import Image
from pathlib import Path
from collections import OrderedDict,namedtuple
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('/content/')
tflite_model = converter.convert()

with open('/content/yolov7_model.tflite', 'wb') as f:
  f.write(tflite_model)# Colab - Convert YOLOv7 weight to TFLite for Android App

# 6. Download the TFLite from Google Colab